**This notebook is the executable version of lab note 3.
It answers the following questions:**

Finally, we answer 4 items:

    1. Are successful creators more connected to high-outdegree users than do unsuccessful creators?
    2. Are mavens more connected to successful creators than to unsuccessful creators?
    3. Do successful creators send more non-follow actions towards mavens than to zombies?
    4. Do successful creators send more non-follow actions towards mavens than to stars?

    

In [1]:
#Run parameters
#used to control every run. Can be user to perfom sensitivity checks
path_dir = r"/Users/../Volumes/Raw/"

low_success = 0.5 #below the median: unsuccessful
high_success = 0.9 #top 10% creators with more followers are deemed successful

low_user_outdegree = 0.25 
high_user_outdegree = 0.75
low_user_activity = 0.25 
high_user_activity = 0.75 

activity_filter = 0

In [2]:
import sys  
import pickle
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
import src.utils
from collections import Counter
from src.utils import import_dta, import_tracks_dta,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob, stripplot_prob,\
reaction_probability, follower_list, filter_quantile, sample_creators_music,\
gen_outbound_creators
import numpy as np
import datetime
import pandas as pd
from tqdm import tqdm
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import os
from statsmodels.stats.proportion import proportions_ztest

In [3]:
def process_date(date):
    '''convert date format like '2013-w09' to '2013-03-04', i.e. the first day of that week'''
    year = date[0:4]
    week = date[6:]
    day = "1"
    date = "{}-{}-1".format(year, week)
    dt = datetime.datetime.strptime(date, "%Y-%W-%w")
    return dt

# Data Imports

We start by importing the raw data.  `follows_sent`, `comments_sent`, `shares_sent`, `likes_sent` and `messages_sent` contains data pn the promotional activities that the 35k users tracked in the dataset directed to other users. It includes the `user_id`, the `fan_id` and the `date_sent` which identifies the date when the prom. activity was sent. `users_info_1st` shows the type of user (creator or non-creator, which is identified by a blank) and the date the user entered the platform, for every user that sent or received prom. activities from any of the 35k users tracked in this dataset, while `users_info` contains the same information, but pertaining to the 35k users themselves.

`follows_received` contains information on the follows received by the 35k users and will be used to generate the successful/unsuccessful groups of content creators.

In [4]:
#affiliations :follows
#favoritings :likes

#used in filtering:
path_dir = r"/Users/../Volumes/Raw/"
tracks = import_tracks_dta(path_dir, "12sample_tracks.dta");

#these are the actions sent to 
follows_sent = import_dta(path_dir, "12sample_affiliations_sent.dta");
comments_sent = import_dta(path_dir, "12sample_comments_made.dta");
shares_sent = import_dta(path_dir, "12sample_reposts_made.dta");
likes_sent = import_dta(path_dir, "12sample_favoritings_made.dta");
messages_sent = import_dta(path_dir, "12sample_messages_sent.dta");

#Used to track information on the 1st degree connections
user_info_1st = import_dta(path_dir, "12sample_1st_deg_user_infos.dta");
user_info_1st.columns = ['user_id', 'type', 'entered_platform'];
user_info = import_dta(path_dir, "12sample_user_infos.dta");

#Used to compute creator's success measure
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta");

%%%%%%%%%% 12sample_tracks.dta %%%%%%%%%%
(56262, 7)
%%%%%%%%%% 12sample_affiliations_sent.dta %%%%%%%%%%
(800913, 3)
%%%%%%%%%% 12sample_comments_made.dta %%%%%%%%%%
(29258, 4)
%%%%%%%%%% 12sample_reposts_made.dta %%%%%%%%%%
(179329, 4)
%%%%%%%%%% 12sample_favoritings_made.dta %%%%%%%%%%
(527701, 4)
%%%%%%%%%% 12sample_messages_sent.dta %%%%%%%%%%
(11091, 3)
%%%%%%%%%% 12sample_1st_deg_user_infos.dta %%%%%%%%%%
(670746, 3)
%%%%%%%%%% 12sample_user_infos.dta %%%%%%%%%%
(35000, 3)
%%%%%%%%%% 12sample_affiliations_received.dta %%%%%%%%%%
(432503, 3)


Indegree and outdegree information.

The function below import the outdegree dataset. Because the raw version of those dataset are too large to be processed in memory, we preprocessed them in a separate script.

In [5]:
# Aggregates preprocessed outdegree of 1st degree users
def import_outdegree(path='/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/'):
    d = {}
    for i in range(6):
       d[str(i)] = pd.read_pickle(os.path.join(path,'{}.pkl'.format(i))) 
       d[str(i)]['created_at'] =  pd.to_datetime(d[str(i)]['created_at'])
       d[str(i)]['created_at'] = pd.to_datetime(d[str(i)]['created_at']).dt.floor('d')
       d[str(i)] = d[str(i)].groupby(['sender_id', 'created_at'], as_index = False).size() 
    
    data_outdegree = pd.concat([d['0'], d['1'], d['2'], d['3'], d['4'], d['5']])
    #data_outdegree.set_index('created_at', inplace = True)
    return data_outdegree

In [6]:
data_outdegree = import_outdegree()

In [7]:
#data_outdegree = data_outdegree.groupby(['sender_id','created_at'], as_index = False).size()

# Preprocessing

## Creator ids, successful and unsucessful creators

Next, we define three lists of ids: one with the ids from the content creators, according to the `users_info` table, one with the ids of successful creators and the last one with the ids of the unsuccessful ones.

Let's start with a list of the id of creators. We also create a dataset with containing information on creators only.

In [8]:
mask = (tracks.track_available == 1) & (tracks.public == 't')
creator_ids = tracks[mask].user_id.unique()

creators = tracks[(tracks.track_available == 1) & (tracks.public == 't')]

#mask = user_info.type == 'creator'
#creator_ids = user_info[mask].user_id.unique()

#creators = user_info[user_info.type == 'creator']

## Putting together a dataset with the promotional activities made by content creators.

The function `gen_actions_sent_df` creates a dataframe with all the promotional activities that content creators sent to users.

In [9]:
def gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent, messages_sent, creator_ids = creator_ids):
    '''
    Creates dataframe containing the actions that content creators send to users.
        Attributes:
                    follows_sent:  dataframe with the follows sent by the 35k users.
                    shares_sent:   dataframe with the shares sent by the 35k users.
                    likes_sent:    dataframe with the likes sent by the 35k users.
                    comments_sent: dataframe with the comments sent by the 35k users.
                    messages_sent: dataframe with the messages sent by the 35k users.
                    creator_ids:   list with content creator ids. If not none, is used to
                                   filter out activities from non creators.
    '''
    
    follows_sent['outbound_activity'] = 'follow'
    follows_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'song_id' in shares_sent.columns:
        shares_sent.drop(columns=["song_id"])
    shares_sent = shares_sent[['reposter_id', "owner_id", 'created_at']]
    shares_sent['outbound_activity'] = 'share'
    shares_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in likes_sent.columns:
        likes_sent.drop(columns=["track_id"], inplace=True)
    likes_sent['outbound_activity'] = 'like'
    likes_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in comments_sent.columns:
        comments_sent.drop(columns=["track_id"], inplace=True)
    comments_sent['outbound_activity'] = 'comment'
    comments_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    messages_sent["outbound_activity"] = 'message'
    messages_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']
    df = pd.concat([follows_sent, shares_sent, likes_sent, comments_sent, messages_sent])


    if type(creator_ids) == numpy.ndarray:
        df = df[df['user_id'].isin(creator_ids)]
        
    df['week_yr'] = df.date_sent.dt.strftime('%Y-w%U')
    df = df.loc[df['user_id'] != df['fan_id'],:]

    return df

In [10]:
actions_sent = gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent,
                                     messages_sent, creator_ids = None)

In [11]:
active_users_ids = actions_sent.groupby('user_id', as_index = False).size()
mask = active_users_ids['size']>= activity_filter
active_users_ids = active_users_ids[mask].user_id.unique()

In [12]:
def successful_creators_followers(follows_received, base_date = datetime.datetime(2016, 5, 30, 0, 0), perc1 = None, perc2 = None, subset_creators = None):
    '''Classifies content creators in successful or unsuccessfull
        Arguments:
                    follows_received: dataframe containing the follows received by content creators
                    base date:        date, in datetime.datetime(YYYY, M, DD, H, M) format, in which the number 
                                      of followers per creator is calculated.
                    perc1:            the threshold used to classify unsuccessful content creators. Creator having 
                                      total followers below the number dictated by this threshold, at the base date,
                                      are classified as unsuccessful 
                    perc2:            the threshold used to classify successful content creators. Creator having 
                                      total followers above the number dictated by this threshold, at the base date,
                                      are classified as successful
                    subset_creators:  a pd.DataFrame containing the creators. If is it available, it will be used to 
                                      filter out non creators and to make sure creators with 0 followers are part of
                                      the resulting dataset.
        
    '''
    print(base_date)

    if 'inbound_activity' not in follows_received.columns:
        follows_received.columns = ['fan_id', 'user_id', 'date_sent']

    mask = (follows_received['date_sent'] < base_date)

    df = follows_received[mask].groupby('user_id', as_index=False).agg({'fan_id': pd.Series.nunique})
    df.columns = ['user_id', 'followers']

    
    if type(subset_creators) == pd.DataFrame:
        print('subsetting...')
        df.set_index('user_id', inplace = True)
        df = df.reindex(subset_creators.user_id.unique())
        df.fillna(0, inplace = True)
        df.reset_index(inplace = True)
        df.columns = ['user_id', 'followers']
        
    mask = df.user_id.isin(active_users_ids)
    df = df[mask]

    low = np.quantile(df.followers, perc1)
    high = np.quantile(df.followers, perc2)

    print("High influencer boundary: {}".format(high))
    print("Low influencer boundary: {}".format(low))

    mask = (df["followers"] <= low) | (df["followers"] >= high)
    
    unsuccessful_creator_ids = df.loc[df["followers"] <= low].user_id.unique()
    successful_creator_ids = df.loc[df["followers"] >= high].user_id.unique()

    return unsuccessful_creator_ids, successful_creator_ids


In [13]:
unsuccessful_ids, successful_ids = successful_creators_followers(follows_received, 
                                                        perc1 = low_success, perc2 = high_success, subset_creators = creators)


2016-05-30 00:00:00
subsetting...
High influencer boundary: 81.0
Low influencer boundary: 13.0


In [14]:
print(len(unsuccessful_ids))
print(len(successful_ids))

1959
389


In [15]:
creators.user_id.nunique()

4604

## Filter only actions that were sent to non-fans

We merge the `actions_sent` dataset with a table containing the date each fan started following the creator.

In [16]:
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follows_received[["fan_id", "user_id", "date_sent"]]
followers.columns = ["fan_id", "user_id", "follower_since"]

actions_sent = actions_sent.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')

Since we are interested in acquisition campaings, we need to produce a dataset that exclude actions targetting fans.
We do that using filters based on the date of the action and the date that the user became a fan of the content creator. The resulting dataframe is named `actions_sent_non_fans`.

We then filter only actions that happened before the user follows the content creator.

In [17]:
mask = (actions_sent.date_sent < actions_sent.follower_since) | (actions_sent.follower_since.isnull())
actions_sent_non_fans =  actions_sent[mask]
actions_sent_non_fans['week_yr_date'] = actions_sent_non_fans.week_yr.apply(lambda x: process_date(x))

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_38194/3855957735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_non_fans['week_yr_date'] = actions_sent_non_fans.week_yr.apply(lambda x: process_date(x))


## Outdegree level

Originally, we only have outdegree information on users that follow at least one user. The function below inputs an outdegree of 0 to users that are following anyone.

In [18]:
#CHANGE DOCUMENTATION
def outdegree_level(data, date, user_info = user_info_1st):
    
    '''
    This function returns the membership table at date equals `date`. Every user that interacted with the 35k tracked 
    users and entered the platform before `date` is present in the table, even if it has indegree 0.
    arguments:
              data:           the indegree dataset.
              user_info:      the dataset containing all the users that interacted with the 35k users tracked.
    '''
    
    data = data[data.created_at.dt.floor('d')<=date]
    data = data.groupby('sender_id').agg({'size':'sum'}).compute()
    
    #merge with user info to obtain users that are not followed by anyone at the current date
    data = user_info_1st.merge(data, left_on = 'user_id', right_on = 'sender_id', how= 'outer')
    data.loc[data['size'].isnull(), 'size'] = 0
    data = data[['user_id', 'size', 'entered_platform']].set_index('user_id')
    
    #filter out users that didnt exist in the current date
    mask = data['entered_platform'].dt.floor('d') <= date
    data = data.loc[mask]
    
    mask = (data['size']>0)
    data.loc[~mask, 'size'] = 0 
    
    return data

In [19]:
#dask is a python api with objects optimized for big data (user directed acyclic graphs). 
dask_outdegree = dd.from_pandas(data_outdegree, npartitions = 3)

In [20]:
last_day =  max(actions_sent.date_sent.dt.floor('d').unique())
user_outdegree = outdegree_level(dask_outdegree, last_day, user_info = user_info_1st)

Now we classify the creator in successful and unsuccessful, according to the threshold defined in the beggining of this notebook.

In [21]:
def user_outdegree_level(oudegree_data, perc1 = None, perc2 = None):
    '''Classifies content creators in successful or unsuccessfull
        Arguments:
                    oudegree_data:    dataframe containing the fans followers at 
                    perc1:            the threshold used to classify unsuccessful content creators. Creator having 
                                      total followers below the number dictated by this threshold, at the base date,
                                      are classified as unsuccessful 
                    perc2:            the threshold used to classify successful content creators. Creator having 
                                      total followers above the number dictated by this threshold, at the base date,
                                      are classified as successful
    '''

    df = oudegree_data.reset_index().iloc[:,:2]
    df.columns = ['user_id', 'followers']

    low = np.quantile(df.followers, perc1)
    high = np.quantile(df.followers, perc2)

    print("High outdegree boundary: {}".format(high))
    print("Low outdegree boundary: {}".format(low))
    
    low_outdegree_ids = df.loc[df["followers"] <= low].user_id.unique()
    high_outdegree_ids = df.loc[df["followers"] >= high].user_id.unique()
    
    
    df['outdegree_level'] = df.user_id.apply(
        lambda x: 'high' if x in high_outdegree_ids else ('low' if x in low_outdegree_ids else None))

    
    return df, low_outdegree_ids, high_outdegree_ids

In [22]:
user_outdegree_df, low_outdegree_ids, high_outdegree_ids = user_outdegree_level(user_outdegree,
                perc1 = low_user_outdegree, perc2 = high_user_outdegree)

High outdegree boundary: 162.0
Low outdegree boundary: 17.0


In the cell below, we create a list with unique ids from users that appear in the oudegree level table. This will later be uses to construct a flow-chart indicating how we lose data based on filters and operations.

In [23]:
receiver_ids = user_outdegree.index.unique()

## Non-follow Actions level

The activity level is defined as the number of actions performed by users. It is important to notice that we only observe actions targeting the 35k users that joined in march 2012. We consider this measure a proxy for the real activity level.

Let's begin by creating a dataset with all action received by those 35k users.

In [24]:
comments_received = import_dta(path_dir, "12sample_comments_received.dta");
shares_received = import_dta(path_dir, "12sample_reposts_received.dta");
likes_received = import_dta(path_dir, "12sample_favoritings_received.dta");
messages_received = import_dta(path_dir, "12sample_messages_received.dta");

if 'song_id' in shares_received:
        shares_received.drop(columns=["song_id"])
shares_received = shares_received[['reposter_id', "owner_id", 'created_at']]
shares_received['inbound_activity'] = 'share'
shares_received.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in likes_received:
        likes_received = likes_received.drop(columns=["track_id"])
likes_received['inbound_activity'] = 'like'
likes_received.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in comments_received:
        comments_received = comments_received.drop(columns=["track_id"])
comments_received['inbound_activity'] = 'comment'
comments_received.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

messages_received["outbound_activity"] = 'message'
messages_received.columns = ['user_id', 'fan_id', 'date_sent', 'inbound_activity']

user_activity_data_35k = pd.concat([shares_received, likes_received, comments_received, messages_received])

%%%%%%%%%% 12sample_comments_received.dta %%%%%%%%%%
(21386, 4)
%%%%%%%%%% 12sample_reposts_received.dta %%%%%%%%%%
(83013, 4)
%%%%%%%%%% 12sample_favoritings_received.dta %%%%%%%%%%
(286903, 4)
%%%%%%%%%% 12sample_messages_received.dta %%%%%%%%%%
(17364, 3)


In [25]:
path_dir_2 = r'/Users/../Volumes/Alter_outbound_activities/'

comments_received = import_dta(path_dir_2, "12sample_1st_deg_comments_made.dta");
shares_received = import_dta(path_dir_2, "12sample_1st_deg_reposts_made.dta");
likes_received = import_dta(path_dir_2, "12sample_1st_deg_favoritings_made.dta");
messages_received = import_dta(path_dir_2, "12sample_1st_deg_messages_sent.dta");

%%%%%%%%%% 12sample_1st_deg_comments_made.dta %%%%%%%%%%
(21463011, 4)
%%%%%%%%%% 12sample_1st_deg_reposts_made.dta %%%%%%%%%%
(18953640, 4)
%%%%%%%%%% 12sample_1st_deg_favoritings_made.dta %%%%%%%%%%
(86793370, 4)
%%%%%%%%%% 12sample_1st_deg_messages_sent.dta %%%%%%%%%%
(16824074, 3)


In [26]:
if 'song_id' in shares_received:
        shares_received.drop(columns=["song_id"])
shares_received = shares_received[['reposter_id', "owner_id", 'created_at']]
shares_received['inbound_activity'] = 'share'
shares_received.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in likes_received:
        likes_received = likes_received.drop(columns=["track_id"])
likes_received['inbound_activity'] = 'like'
likes_received.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in comments_received:
        comments_received = comments_received.drop(columns=["track_id"])
comments_received['inbound_activity'] = 'comment'
comments_received.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

messages_received["outbound_activity"] = 'message'
messages_received.columns = ['user_id', 'fan_id', 'date_sent', 'inbound_activity']

In [27]:
user_activity_data = pd.concat([shares_received, likes_received, comments_received, messages_received])

In [28]:
def user_activity_level(user_activity_data, receiver_ids, perc1 = None, perc2 = None):
    '''Classifies users based on the amount of non-follow activities
        Arguments:
                    user_activity_data:   dataframe containing the user activities 
                    base date:             date, in datetime.datetime(YYYY, M, DD, H, M) format, in which the number 
                                           of followers per creator is calculated.
                    perc1:                 the threshold used to classify unsuccessful content creators. Creator having 
                                           total followers below the number dictated by this threshold, at the base date,
                                           are classified as unsuccessful 
                    perc2:                 the threshold used to classify successful content creators. Creator having 
                                           total followers above the number dictated by this threshold, at the base date,
                                           are classified as successful
    '''

    df = user_activity_data.groupby('fan_id', as_index = True).size()
    df = df.reindex(receiver_ids)
    print(df.shape)
    df = df.reset_index()
    df.columns = ['user_id', 'activities_performed']
    
    df.loc[df.activities_performed.isna(), 'activities_performed'] = 0
    #classification should happen after this

    low = np.quantile(df.activities_performed, perc1)
    high = np.quantile(df.activities_performed, perc2)

    print("High activity boundary: {}".format(high))
    print("Low activity boundary: {}".format(low))
    
    low_activity_ids = df.loc[df["activities_performed"] <= low].user_id.unique()
    high_activity_ids = df.loc[df["activities_performed"] > high].user_id.unique()
    
    df['activity_level'] = df.user_id.apply(
    lambda x: 'high' if x in high_activity_ids else ('low' if x in low_activity_ids else None))

    return df, low_activity_ids, high_activity_ids

In [29]:
activity_level, low_activity_ids, high_activity_ids = user_activity_level(user_activity_data, receiver_ids, 
                perc1 = low_user_activity, perc2 = high_user_activity)

(670746,)
High activity boundary: 85.0
Low activity boundary: 1.0


Once more we create an object containing the unique ids of users in the resulting dataset. This will be used in a flow-chart, as explained.

In [30]:
activity_ids = np.append(low_activity_ids, high_activity_ids)

In [31]:
len(set(activity_ids).intersection(set(receiver_ids)))
#only 35493 from 240292 that follow at least one of the 35k, performed at least one non-follow action

360347

# Analysis

Now we merge the datasets with the outdegree, and non-follow activities information.

In [32]:
table_data = user_outdegree_df.merge(activity_level, left_on = 'user_id', right_on = 'user_id', how = 'inner')

In [33]:
table_data.groupby(['outdegree_level', 'activity_level']).size()

outdegree_level  activity_level
high             high              98464
                 low                7790
low              high               7266
                 low               85480
dtype: int64

And, finally, we create the 4 user groups that we are interested in: *Maven*, *Zombie*, *Stars* and *Stalker*. Everyone else is classified as *Other*.

In [34]:
table_data['Type'] = table_data.apply(lambda x: 
    'Maven' if (x.outdegree_level == 'low') & (x.activity_level == 'high')
     else ('Zombie' if (x.outdegree_level == 'low') & (x.activity_level == 'low')
     else ('Stalker' if (x.outdegree_level == 'high') & (x.activity_level == 'low')
     else ('Star' if (x.outdegree_level == 'high') & (x.activity_level == 'high')
     else 'other'))), axis=1)

Let's keep track of each user type's ids, as it gives us a compact way of identifying each user.

In [35]:
zombie_ids = table_data.loc[table_data.Type =='Zombie'].user_id.unique()
star_ids = table_data.loc[table_data.Type =='Star'].user_id.unique()
maven_ids = table_data.loc[table_data.Type =='Maven'].user_id.unique()
stalker_ids = table_data.loc[table_data.Type =='Stalker'].user_id.unique()

actions_sent_non_fans['user_type'] = actions_sent_non_fans.fan_id.apply(lambda x: 'Maven' if x in maven_ids else 
                          ('Zombie' if x in zombie_ids else
                          ('Stalker' if x in stalker_ids else
                          ('Star' if x in star_ids else 'other'))))

## classify content creators
actions_sent_non_fans['creator_type'] = actions_sent_non_fans.user_id.apply(
                                            lambda x: 'successful' if x in successful_ids else 
                                            ('unsuccessful' if x in unsuccessful_ids else 'other'))

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_38194/2520098529.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_non_fans['user_type'] = actions_sent_non_fans.fan_id.apply(lambda x: 'Maven' if x in maven_ids else
/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_38194/2520098529.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_non_fans['creator_type'] = actions_sent_non_fans.user_id.apply(


In [36]:
table_data.Type.value_counts()

other      471746
Star        98464
Zombie      85480
Stalker      7790
Maven        7266
Name: Type, dtype: int64

## Table 2

In [37]:
#incoming follows
follows_sent.merge(table_data, left_on = 'fan_id', right_on = 'user_id', how = 'inner')\
.groupby('Type').size()

Type
Maven       67740
Stalker      3510
Star       135128
Zombie      92632
other      488454
dtype: int64

In [38]:
#outgoing follows
follows_received.merge(table_data, left_on = 'fan_id', right_on = 'user_id', how = 'inner')\
.groupby('Type').size()

Type
Maven        1851
Stalker      7215
Star        66871
Zombie      14548
other      328892
dtype: int64

In [39]:
#incoming nonfollow actions
mask = actions_sent.outbound_activity != 'follow'
incoming_non_follow = actions_sent[mask]
incoming_non_follow.merge(table_data, left_on = 'fan_id', right_on = 'user_id', how = 'inner')\
.groupby('Type').size()

Type
Maven       98150
Stalker       143
Star       143050
Zombie      12009
other      311495
dtype: int64

In [40]:
#outgoing nonfollow actions
user_activity_data_35k.merge(table_data, left_on = 'fan_id', right_on = 'user_id', how = 'inner')\
.groupby('Type').size()

Type
Maven       3839
Stalker       21
Star       53979
Zombie       622
other      55951
dtype: int64

In [41]:
#mavens
67740+61356+98150+3754226

3981472

In [42]:
#star
135128+83529227+143050+99760233

183567638

In [43]:
#stalker
3510+3531262+143+2754

3537669

In [44]:
#zombie
92632+245300+12009+20076

370017

# Testing

Finally, we answer 4 items:

    1. Are successful creators more connected to high-outdegree users than do unsuccessful creators?
    2. Are mavens more connected to successful creators thanto unsuccessful creators?
    3. Do successful creators send more non-follow actions towards mavens than to zombies?
    4. Do successful creators send more non-follow actions towards mavens than to stars?

## Desc. Statistics

Let's start with 3, since if provide the figures we are going to use to answer to other items:

### 3. Descriptive statistics of the four groups.

In [45]:
table_data.groupby('Type', as_index = False).activities_performed.describe().unstack(1)

,0,1,2,3,4
Type,Maven,Stalker,Star,Zombie,other
count,7266.0,7790.0,98464.0,85480.0,471746.0
mean,516.684008,0.35353,1013.164537,0.234862,59.438395
std,4048.447866,0.478096,3579.971524,0.423915,454.138688
min,86.0,0.0,86.0,0.0,0.0
25%,111.0,0.0,182.0,0.0,2.0
50%,155.0,0.0,386.0,0.0,10.0
75%,254.0,1.0,921.0,0.0,44.0
max,147955.0,1.0,403169.0,1.0,86358.0


### 3.1. Are successful creators more connected to high-outdegree users than do unsuccessful creators?


We want to measure if successful creators are skilled in identifying *maven* and connecting to *high-outdegree and mavens*. Unfortunatelly,  we cannot use a probability measure to do the comparison, since it is sharply affected by group size variations. One measure that sidestep this issue is the odds ratio, a statistic that is better suited to compare groups of different sizes because it takes the number of events and non events in each group into account.

To calculate the odds ratio of for a given user type, say *hardcore*, we need the probability that a *hardcore* user is targeted by a successful creator. That is obtained by the following ratio: `p_maven_users` = `total unique users of type maven target by successful creators`/`total unique users of type maven`. 

We then use it to calculate: `odds_maven_user` = `p_maven_users`/(1-`p_maven_users`)

Finally, let's say we want to compare `maven` and `zombie` users. We would then use the meetric `odds_ratio` = `odds_maven_user`/`odds_zombie_user`

We will also report the Confidence Intervals (95%) for the Odds Ratio. The function below automatizes that calculation.

In [46]:
import numpy as np
import scipy.stats as stats

def calculate_odds_ratio_ci(exposed_cases, exposed_controls, nonexposed_cases, nonexposed_controls):
    # Calculate the odds ratio (OR)
    odds_ratio = (exposed_cases / exposed_controls) / (nonexposed_cases / nonexposed_controls)

    # Calculate the 95% confidence interval (CI) for the OR
    log_odds_ratio = np.log(odds_ratio)
    std_error = np.sqrt(1/exposed_cases + 1/exposed_controls + 1/nonexposed_cases + 1/nonexposed_controls)
    z_score = stats.norm.ppf(0.975)
    lower_ci = np.exp(log_odds_ratio - z_score*std_error)
    upper_ci = np.exp(log_odds_ratio + z_score*std_error)

    return round(odds_ratio, 2), (round(lower_ci, 2), round(upper_ci, 2))

In [47]:
from scipy.stats import chi2_contingency

In [48]:
table_data.columns = ['fan_id', 'followers', 'outdegree_level', 'activities_performed', 'activity_level', 'Type']
table = follows_received.merge(table_data, left_on = 'fan_id', right_on = 'fan_id', how = 'inner') #suffixes

table['creator_type'] = table.user_id.apply(
                                            lambda x: 'successful' if x in successful_ids else 
                                            ('unsuccessful' if x in unsuccessful_ids else 'other'))

In [49]:
table1 = table.groupby(['creator_type','outdegree_level'], as_index = False).size()

In [50]:
high_outdegree_ids = table[table.outdegree_level == 'high'].fan_id.unique()

In [51]:
mask = (table1.outdegree_level == 'high') & (table1.creator_type == 'successful')
g1 = table1.loc[mask]['size'].values[0]

mask = (table1.outdegree_level == 'high') & (table1.creator_type == 'unsuccessful')
g2 = max(table1.loc[mask]['size'].values[0],1)
group_ids1 = table.loc[table.creator_type == 'successful']
group_ids2 = table.loc[table.creator_type == 'unsuccessful']

tab = np.array([[g1, g2], [len(group_ids1)-g1, len(group_ids2)-g2]])

# calculate the odds ratio by taking the ratio of the odds of the event occurring in the two groups
odds_ratio = (g1/(len(group_ids1)-g1)/(g2/(len(group_ids2)-g2)))
print("Odds Ratio:", round(odds_ratio,4))

# perform a chi-square test to determine whether the observed odds ratio is statistically significant
chi2, p_value, _, _ = chi2_contingency(tab)
print("Chi-Square Statistic:", round(chi2,4))
print("P-Value:", round(p_value,4))

Odds Ratio: 3.1963
Chi-Square Statistic: 2332.494
P-Value: 0.0


In [52]:
calculate_odds_ratio_ci(g1, g2, len(group_ids1)-g1, len(group_ids2)-g2)

(3.2, (3.04, 3.36))

In [53]:
print(g1)
print(len(group_ids1)-g1)
print(g2)
print(len(group_ids2)-g2)

58816
50211
2289
6246


### 3.2. Do successful creators acquire more Mavens than unsuccessful creators??


In [54]:
table1 = table.groupby(['creator_type','Type'], as_index = False).size()

In [55]:
table1

,creator_type,Type,size
0,other,Maven,1054
1,other,Stalker,4976
2,other,Star,29905
3,other,Zombie,11080
4,other,other,254800
5,successful,Maven,762
6,successful,Stalker,2116
7,successful,Star,35928
8,successful,Zombie,2793
9,successful,other,67428


In [56]:
mask = (table1.Type == 'Maven') & (table1.creator_type == 'successful')
g1 = table1.loc[mask]['size'].values[0]

mask = (table1.Type == 'Maven') & (table1.creator_type == 'unsuccessful')
g2 = table1.loc[mask]['size'].values[0]
group_ids1 = table.loc[table.creator_type == 'successful']
group_ids2 = table.loc[table.creator_type == 'unsuccessful']

tab = np.array([[g1, g2], [len(group_ids1)-g1, len(group_ids2)-g2]])

# calculate the odds ratio by taking the ratio of the odds of the event occurring in the two groups
odds_ratio = (g1/(len(group_ids1)-g1)/(g2/(len(group_ids2)-g2)))
print("Odds Ratio:", round(odds_ratio,4))

# perform a chi-square test to determine whether the observed odds ratio is statistically significant
chi2, p_value, _, _ = chi2_contingency(tab)
print("Chi-Square Statistic:", round(chi2,4))
print("P-Value:", round(p_value,4))

Odds Ratio: 1.7093
Chi-Square Statistic: 9.3826
P-Value: 0.0022


In [57]:
calculate_odds_ratio_ci(g1, g2, len(group_ids1)-g1, len(group_ids2)-g2)

(1.71, (1.22, 2.4))

In [58]:
print(g1)
print(len(group_ids1)-g1)
print(g2)
print(len(group_ids2)-g2)

762
108265
35
8500


###     3. Do successful creators send more non-follow actions towards mavens than to zombies?

In [59]:
#actions sentdata + mavens and zombie classification + successful/unsuccessful

In [60]:
distribution_target_user_type = actions_sent_non_fans.groupby(['creator_type', 'user_type']).fan_id.nunique()
dist_target_user = distribution_target_user_type.to_frame().reset_index()
dist_target_user.columns = ['creator_type', 'user_type', 'non_follow_actions']

In [61]:
dist_target_user

,creator_type,user_type,non_follow_actions
0,other,Maven,4578
1,other,Stalker,1011
2,other,Star,35742
3,other,Zombie,66601
4,other,other,231597
5,successful,Maven,2293
6,successful,Stalker,461
7,successful,Star,31095
8,successful,Zombie,5109
9,successful,other,59139


In [62]:
mask = (dist_target_user.user_type == 'Maven') & (dist_target_user.creator_type == 'successful')
g1 = dist_target_user.loc[mask].non_follow_actions.values[0]

mask = (dist_target_user.user_type == 'Zombie') & (dist_target_user.creator_type == 'successful')
g2 = dist_target_user.loc[mask].non_follow_actions.values[0]

group_ids1 = maven_ids
group_ids2 = zombie_ids

table = np.array([[g1, g2], [len(group_ids1)-g1, len(group_ids2)-g2]])

# calculate the odds ratio by taking the ratio of the odds of the event occurring in the two groups
odds_ratio = (g1/(len(group_ids1)-g1)/(g2/(len(group_ids2)-g2)))
print("Odds Ratio:", round(odds_ratio,4))

# perform a chi-square test to determine whether the observed odds ratio is statistically significant
chi2, p_value, _, _ = chi2_contingency(table)
print("Chi-Square Statistic:", round(chi2,4))
print("P-Value:", round(p_value,4))

Odds Ratio: 7.2535
Chi-Square Statistic: 5963.7311
P-Value: 0.0


In [63]:
#unsuccessful
mask = (dist_target_user.user_type == 'Maven') & (dist_target_user.creator_type == 'unsuccessful')
g1 = dist_target_user.loc[mask].non_follow_actions.values[0]

mask = (dist_target_user.user_type == 'Zombie') & (dist_target_user.creator_type == 'unsuccessful')
g2 = dist_target_user.loc[mask].non_follow_actions.values[0]

group_ids1 = maven_ids
group_ids2 = zombie_ids

table = np.array([[g1, g2], [len(group_ids1)-g1, len(group_ids2)-g2]])

# calculate the odds ratio by taking the ratio of the odds of the event occurring in the two groups
odds_ratio = (g1/(len(group_ids1)-g1)/(g2/(len(group_ids2)-g2)))
print("Odds Ratio:", round(odds_ratio,4))

# perform a chi-square test to determine whether the observed odds ratio is statistically significant
chi2, p_value, _, _ = chi2_contingency(table)
print("Chi-Square Statistic:", round(chi2,4))
print("P-Value:", round(p_value,4))

Odds Ratio: 2.8919
Chi-Square Statistic: 595.1864
P-Value: 0.0


In [64]:
calculate_odds_ratio_ci(g1, g2, len(group_ids1)-g1, len(group_ids2)-g2)

(2.89, (2.65, 3.16))

In [65]:
print(g1)
print(len(group_ids1)-g1)
print(g2)
print(len(group_ids2)-g2)

649
6617
2804
82676


###     4. Do successful creators send more non-follow actions towards mavens than to stars?

In [66]:
mask = (dist_target_user.user_type == 'Maven') & (dist_target_user.creator_type == 'successful')
g1 = dist_target_user.loc[mask].non_follow_actions.values[0]

mask = (dist_target_user.user_type == 'Star') & (dist_target_user.creator_type == 'successful')
g2 = dist_target_user.loc[mask].non_follow_actions.values[0]

group_ids1 = maven_ids
group_ids2 = star_ids

table = np.array([[g1, g2], [len(group_ids1)-g1, len(group_ids2)-g2]])

# calculate the odds ratio by taking the ratio of the odds of the event occurring in the two groups
odds_ratio = (g1/(len(group_ids1)-g1)/(g2/(len(group_ids2)-g2)))
print("Odds Ratio:", round(odds_ratio,4))

# perform a chi-square test to determine whether the observed odds ratio is statistically significant
chi2, p_value, _, _ = chi2_contingency(table)
print("Chi-Square Statistic:", round(chi2,4))
print("P-Value:", round(p_value,5))

Odds Ratio: 0.999
Chi-Square Statistic: 0.0007
P-Value: 0.97919


In [67]:
#unsuccessful
mask = (dist_target_user.user_type == 'Maven') & (dist_target_user.creator_type == 'unsuccessful')
g1 = dist_target_user.loc[mask].non_follow_actions.values[0]

mask = (dist_target_user.user_type == 'Star') & (dist_target_user.creator_type == 'unsuccessful')
g2 = dist_target_user.loc[mask].non_follow_actions.values[0]

group_ids1 = maven_ids
group_ids2 = star_ids

table = np.array([[g1, g2], [len(group_ids1)-g1, len(group_ids2)-g2]])

# calculate the odds ratio by taking the ratio of the odds of the event occurring in the two groups
odds_ratio = (g1/(len(group_ids1)-g1)/(g2/(len(group_ids2)-g2)))
print("Odds Ratio:", round(odds_ratio,4))

# perform a chi-square test to determine whether the observed odds ratio is statistically significant
chi2, p_value, _, _ = chi2_contingency(table)
print("Chi-Square Statistic:", round(chi2,4))
print("P-Value:", round(p_value,5))

Odds Ratio: 4.8621
Chi-Square Statistic: 1363.6109
P-Value: 0.0


In [68]:
p_value

1.697967597051716e-298

In [69]:
calculate_odds_ratio_ci(g1, g2, len(group_ids1)-g1, len(group_ids2)-g2)

(4.86, (4.43, 5.33))

In [70]:
print(g1)
print(len(group_ids1)-g1)
print(g2)
print(len(group_ids2)-g2)

649
6617
1947
96517


## 5 Are creators better off (more likely to succeed) if they reach out/target mavens than stars?

To answer question 5, we will fit a logistic regression. The goal is to predict if a creator becames succesfull based on the number of non-follow activities and follows sent, both in aggregate and per user type.

Let's create the dataset to be used in the model.

In [71]:
#Create target:
#Per creator:
#1 non-follow activities sent
#2 follow activities sent
#3 non-follow activities sent per user type
#4 follow activities sent per user type

In [72]:
actions_sent_non_fans

,user_id,fan_id,date_sent,outbound_activity,week_yr,follower_since,week_yr_date,user_type,creator_type
0,37825116,61046,2013-03-01 00:14:58,follow,2013-w08,NaT,2013-02-25,Maven,other
1,37825116,80377,2013-03-01 00:14:58,follow,2013-w08,NaT,2013-02-25,other,other
2,37825116,122596,2013-03-01 00:14:58,follow,2013-w08,NaT,2013-02-25,other,other
3,37825116,133935,2013-03-01 00:14:58,follow,2013-w08,NaT,2013-02-25,Star,other
4,37825116,165779,2013-03-01 00:14:58,follow,2013-w08,NaT,2013-02-25,other,other
...,...,...,...,...,...,...,...,...,...
1540534,38393304,21462161,2015-05-30 13:47:04,message,2015-w21,2015-05-31 05:34:55,2015-05-25,other,successful
1540535,38393304,21462161,2015-05-30 14:13:44,message,2015-w21,2015-05-31 05:34:55,2015-05-25,other,successful
1540536,38393304,66073317,2015-06-02 00:01:37,message,2015-w22,NaT,2015-06-01,other,successful
1540542,38445740,118118511,2015-06-15 14:04:10,message,2015-w24,NaT,2015-06-15,Star,successful


model_df = pd.DataFrame(creator_ids, columns = ['creator_id'])
model_df['successful'] = model_df.creator_id.apply(lambda x: 1 if x in successful_ids else 0)

mask = (actions_sent_non_fans.outbound_activity != 'follow')

total_non_follow_activities = actions_sent_non_fans[mask].groupby('user_id', as_index = False).size()
total_non_follow_activities.columns = ['creator_id', 'total_non_follow_activities']

Mavens_targeted_non_follow = actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Maven')].groupby('user_id', as_index = False).size()
Mavens_targeted_non_follow.columns = ['creator_id', 'Mavens_targeted_non_follow']

Zombies_targeted_non_follow =  actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Zombie')].groupby('user_id', as_index = False).size()
Zombies_targeted_non_follow.columns = ['creator_id', 'Zombie_targeted_non_follow']


Stars_targeted_non_follow =  actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Star')].groupby('user_id', as_index = False).size()
Stars_targeted_non_follow.columns = ['creator_id', 'Stars_targeted_non_follow']

Stalkers_targeted_non_follow =  actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Stalker')].groupby('user_id', as_index = False).size()
Stalkers_targeted_non_follow.columns = ['creator_id', 'Stalker_targeted_non_follow']

mask = (actions_sent_non_fans.outbound_activity == 'follow')
model_df['total_follow_activities'] = model_df.creator_id.apply(lambda x: actions_sent_non_fans[mask].\
                                          loc[(mask) & (actions_sent_non_fans.user_id == x)].shape[0])

Mavens_targeted_follow = actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Maven')].groupby('user_id', as_index = False).size()
Mavens_targeted_follow.columns = ['creator_id', 'Mavens_targeted_non_follow']

Zombies_targeted_follow = actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Zombie')].groupby('user_id', as_index = False).size()
Zombies_targeted_follow.columns = ['creator_id', 'Zombie_targeted_non_follow']


Stars_targeted_follow =  actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Star')].groupby('user_id', as_index = False).size()
Stars_targeted_non_follow.columns = ['creator_id', 'Stars_targeted_non_follow']

Stalkers_targeted_follow = actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Stalker')].groupby('user_id', as_index = False).size()
Stalkers_targeted_follow.columns = ['creator_id', 'Stalker_targeted_non_follow']

### 3.5.a: Is p(creator being successful | creator sent a follow to a maven) > p(creator being successful| creator sent a follow to a star)?

In [73]:
#p(A|B) = P(A&B)/P(B)

In [74]:
#1 create column: sent follow to maven

In [75]:
bdf = pd.DataFrame(creator_ids, columns = ['creator_id'])
bdf['successful'] = bdf.creator_id.apply(lambda x: 1 if x in successful_ids else 0)

mask = (actions_sent_non_fans.outbound_activity == 'follow')
Mavens_targeted_follow = actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Maven')].groupby('user_id', as_index = False).size()
Mavens_targeted_follow.columns = ['creator_id', 'Mavens_targeted_follow']

Stars_targeted_follow =  actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Star')].groupby('user_id', as_index = False).size()
Stars_targeted_follow.columns = ['creator_id', 'Stars_targeted_follow']

Zombies_targeted_follow = actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Zombie')].groupby('user_id', as_index = False).size()
Zombies_targeted_follow.columns = ['creator_id', 'Zombies_targeted_follow']

Stalkers_targeted_follow =  actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Stalker')].groupby('user_id', as_index = False).size()
Stalkers_targeted_follow.columns = ['creator_id', 'Stalkers_targeted_follow']

bdf = bdf.merge(Mavens_targeted_follow, left_on = 'creator_id', right_on='creator_id', how = 'left')
bdf = bdf.fillna(0)
bdf = bdf.merge(Stars_targeted_follow, left_on = 'creator_id', right_on='creator_id', how = 'left')
bdf = bdf.fillna(0)
bdf = bdf.merge(Zombies_targeted_follow, left_on = 'creator_id', right_on='creator_id', how = 'left')
bdf = bdf.fillna(0)
bdf = bdf.merge(Stalkers_targeted_follow, left_on = 'creator_id', right_on='creator_id', how = 'left')
bdf = bdf.fillna(0)

bdf['followed_Maven'] = (bdf.Mavens_targeted_follow > 0)*1
bdf['followed_Star'] = (bdf.Stars_targeted_follow > 0)*1
bdf['followed_Zombie'] = (bdf.Zombies_targeted_follow > 0)*1
bdf['followed_Stalker'] = (bdf.Stalkers_targeted_follow > 0)*1

In [76]:
P_A_and_B = bdf.loc[(bdf.successful == 1) & (bdf.followed_Maven == 1)].shape[0]/bdf.shape[0]
P_B = bdf.loc[(bdf.followed_Maven == 1)].shape[0]/bdf.shape[0]
P_success_g_Maven = round(P_A_and_B/P_B,3)
print('P(success|Maven): {}'.format(P_success_g_Maven))

P_A_and_B = bdf.loc[(bdf.successful == 1) & (bdf.followed_Star == 1)].shape[0]/bdf.shape[0]
P_B = bdf.loc[(bdf.followed_Star == 1)].shape[0]/bdf.shape[0]
P_success_g_Star = round(P_A_and_B/P_B,3)
print('P(success|Star): {}'.format(P_success_g_Star))

P_A_and_B = bdf.loc[(bdf.successful == 1) & (bdf.followed_Zombie == 1)].shape[0]/bdf.shape[0]
P_B = bdf.loc[(bdf.followed_Zombie == 1)].shape[0]/bdf.shape[0]
P_success_g_Zombie = round(P_A_and_B/P_B,3)
print('P(success|Zombie): {}'.format(P_success_g_Zombie))

P_A_and_B = bdf.loc[(bdf.successful == 1) & (bdf.followed_Stalker == 1)].shape[0]/bdf.shape[0]
P_B = bdf.loc[(bdf.followed_Stalker == 1)].shape[0]/bdf.shape[0]
P_success_g_Stalker = round(P_A_and_B/P_B,3)
print('P(success|Stalker): {}'.format(P_success_g_Stalker))

P(success|Maven): 0.145
P(success|Star): 0.147
P(success|Zombie): 0.137
P(success|Stalker): 0.411


In [77]:
#P(A)
bdf.loc[(bdf.successful == 1)].shape[0]/bdf.shape[0]
#P(B)
bdf.loc[(bdf.followed_Maven == 1)].shape[0]/bdf.shape[0]
#P(C)
bdf.loc[(bdf.followed_Star == 1)].shape[0]/bdf.shape[0]

#P(A&B)
bdf.loc[(bdf.successful == 1)&(bdf.followed_Maven == 1)].shape[0]/bdf.shape[0]
#P(A&C)
bdf.loc[(bdf.successful == 1)&(bdf.followed_Star == 1)].shape[0]/bdf.shape[0]


0.07276281494352736

In [78]:
mask = (actions_sent_non_fans.outbound_activity != 'follow')
Mavens_targeted_non_follow = actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Maven')].groupby('user_id', as_index = False).size()
Mavens_targeted_non_follow.columns = ['creator_id', 'Mavens_targeted_non_follow']

Stars_targeted_non_follow =  actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Star')].groupby('user_id', as_index = False).size()
Stars_targeted_non_follow.columns = ['creator_id', 'Stars_targeted_non_follow']

Zombies_targeted_non_follow = actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Zombie')].groupby('user_id', as_index = False).size()
Zombies_targeted_non_follow.columns = ['creator_id', 'Zombies_targeted_non_follow']

Stalkers_targeted_non_follow =  actions_sent_non_fans[(mask)
                & (actions_sent_non_fans.user_type == 'Stalker')].groupby('user_id', as_index = False).size()
Stalkers_targeted_non_follow.columns = ['creator_id', 'Stalkers_targeted_non_follow']

bdf = bdf.merge(Mavens_targeted_non_follow, left_on = 'creator_id', right_on='creator_id', how = 'left')
bdf = bdf.fillna(0)
bdf = bdf.merge(Stars_targeted_non_follow, left_on = 'creator_id', right_on='creator_id', how = 'left')
bdf = bdf.fillna(0)
bdf = bdf.merge(Zombies_targeted_non_follow, left_on = 'creator_id', right_on='creator_id', how = 'left')
bdf = bdf.fillna(0)
bdf = bdf.merge(Stalkers_targeted_non_follow, left_on = 'creator_id', right_on='creator_id', how = 'left')
bdf = bdf.fillna(0)

bdf['non_follow_Maven'] = (bdf.Mavens_targeted_non_follow > 0)*1
bdf['non_follow_Star'] = (bdf.Stars_targeted_non_follow > 0)*1
bdf['non_follow_Zombie'] = (bdf.Zombies_targeted_non_follow > 0)*1
bdf['non_follow_Stalker'] = (bdf.Stalkers_targeted_non_follow > 0)*1

In [79]:
#P(D)
bdf.loc[(bdf.non_follow_Maven == 1)].shape[0]/bdf.shape[0]
#P(E)
bdf.loc[(bdf.non_follow_Star == 1)].shape[0]/bdf.shape[0]

#P(A&D)
bdf.loc[(bdf.successful == 1)&(bdf.non_follow_Maven == 1)].shape[0]/bdf.shape[0]
#P(A&E)
bdf.loc[(bdf.successful == 1)&(bdf.non_follow_Star == 1)].shape[0]/bdf.shape[0]

0.059296264118158123

In [80]:
P_A_and_B = bdf.loc[(bdf.successful == 1) & (bdf.non_follow_Maven == 1)].shape[0]/bdf.shape[0]
P_B = bdf.loc[(bdf.non_follow_Maven == 1)].shape[0]/bdf.shape[0]
P_success_g_Maven = round(P_A_and_B/P_B,3)
print('P(success|Maven): {}'.format(P_success_g_Maven))

P_A_and_B = bdf.loc[(bdf.successful == 1) & (bdf.non_follow_Star == 1)].shape[0]/bdf.shape[0]
P_B = bdf.loc[(bdf.non_follow_Star == 1)].shape[0]/bdf.shape[0]
P_success_g_Star = round(P_A_and_B/P_B,3)
print('P(success|Star): {}'.format(P_success_g_Star))

P_A_and_B = bdf.loc[(bdf.successful == 1) & (bdf.non_follow_Zombie == 1)].shape[0]/bdf.shape[0]
P_B = bdf.loc[(bdf.non_follow_Zombie == 1)].shape[0]/bdf.shape[0]
P_success_g_Zombie = round(P_A_and_B/P_B,3)
print('P(success|Zombie): {}'.format(P_success_g_Zombie))

P_A_and_B = bdf.loc[(bdf.successful == 1) & (bdf.non_follow_Stalker == 1)].shape[0]/bdf.shape[0]
P_B = bdf.loc[(bdf.non_follow_Stalker == 1)].shape[0]/bdf.shape[0]
P_success_g_Stalker= round(P_A_and_B/P_B,3)
print('P(success|Stalker): {}'.format(P_success_g_Stalker))

P(success|Maven): 0.207
P(success|Star): 0.211
P(success|Zombie): 0.214
P(success|Stalker): 0.5


In [81]:
P_B

0.0047784535186794095

In [82]:
total = len(maven_ids)+len(zombie_ids)+len(stalker_ids)+len(star_ids)
print(len(maven_ids)/total*100)
print(len(zombie_ids)/total*100)
print(len(stalker_ids)/total*100)
print(len(star_ids)/total*100)

3.651256281407035
42.95477386934674
3.9145728643216082
49.47939698492462


In [83]:
len(successful_ids)

389

In [84]:
user_outdegree_df['user_type'] = user_outdegree_df.user_id.apply(lambda x: 'Maven' if x in maven_ids else 
                          ('Zombie' if x in zombie_ids else
                          ('Stalker' if x in stalker_ids else
                          ('Star' if x in star_ids else 'other'))))

In [85]:
user_outdegree_df.groupby('user_type').agg({'followers': sum})

,followers
user_type,
Maven,61356.0
Stalker,3531262.0
Star,83529227.0
Zombie,245300.0
other,71722663.0


In [86]:
table_data

,fan_id,followers,outdegree_level,activities_performed,activity_level,Type
0,706,549.0,high,185.0,high,Star
1,845,43.0,None,51.0,None,other
2,1255,429.0,high,541.0,high,Star
3,1506,455.0,high,859.0,high,Star
4,2667,408.0,high,251.0,high,Star
...,...,...,...,...,...,...
670741,160366917,62.0,None,0.0,low,other
670742,160556475,50.0,None,0.0,low,other
670743,160592014,50.0,None,0.0,low,other
670744,160615335,50.0,None,0.0,low,other
